Reference: https://www.kaggle.com/code/venkatasubramanian/automatic-ticket-classification-notebook#Topic-Modelling-using-NMF

In [1]:
import pandas as pd
import json
import numpy as np
import re, nltk
nltk.download('averaged_perceptron_tagger')
import spacy
nlp = spacy.load("en_core_web_sm")
# import en_core_web_sm
# nlp = en_core_web_sm.load()
from textblob import TextBlob

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\dxlee\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


# Data Loading

In [2]:
df = pd.json_normalize(json.load(open("complaints-2021-05-14_08_16_.json")))

# Basic Data Understanding

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78313 entries, 0 to 78312
Data columns (total 22 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   _index                             78313 non-null  object 
 1   _type                              78313 non-null  object 
 2   _id                                78313 non-null  object 
 3   _score                             78313 non-null  float64
 4   _source.tags                       10900 non-null  object 
 5   _source.zip_code                   71556 non-null  object 
 6   _source.complaint_id               78313 non-null  object 
 7   _source.issue                      78313 non-null  object 
 8   _source.date_received              78313 non-null  object 
 9   _source.state                      76322 non-null  object 
 10  _source.consumer_disputed          78313 non-null  object 
 11  _source.product                    78313 non-null  obj

In [4]:
df.head()

,_index,_type,_id,_score,_source.tags,_source.zip_code,_source.complaint_id,_source.issue,_source.date_received,_source.state,...,_source.company_response,_source.company,_source.submitted_via,_source.date_sent_to_company,_source.company_public_response,_source.sub_product,_source.timely,_source.complaint_what_happened,_source.sub_issue,_source.consumer_consent_provided
0,complaint-public-v2,complaint,3211475,0.0,None,90301,3211475,Attempts to collect debt not owed,2019-04-13T12:00:00-05:00,CA,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-04-13T12:00:00-05:00,None,Credit card debt,Yes,,Debt is not yours,Consent not provided
1,complaint-public-v2,complaint,3229299,0.0,Servicemember,319XX,3229299,Written notification about debt,2019-05-01T12:00:00-05:00,GA,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-05-01T12:00:00-05:00,None,Credit card debt,Yes,Good morning my name is XXXX XXXX and I apprec...,Didn't receive enough information to verify debt,Consent provided
2,complaint-public-v2,complaint,3199379,0.0,None,77069,3199379,"Other features, terms, or problems",2019-04-02T12:00:00-05:00,TX,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-04-02T12:00:00-05:00,None,General-purpose credit card or charge card,Yes,I upgraded my XXXX XXXX card in XX/XX/2018 and...,Problem with rewards from credit card,Consent provided
3,complaint-public-v2,complaint,2673060,0.0,None,48066,2673060,Trouble during payment process,2017-09-13T12:00:00-05:00,MI,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2017-09-14T12:00:00-05:00,None,Conventional home mortgage,Yes,,None,Consent not provided
4,complaint-public-v2,complaint,3203545,0.0,None,10473,3203545,Fees or interest,2019-04-05T12:00:00-05:00,NY,...,Closed with explanation,JPMORGAN CHASE & CO.,Referral,2019-04-05T12:00:00-05:00,None,General-purpose credit card or charge card,Yes,,Charged too much interest,N/A


In [5]:
df.tail()

,_index,_type,_id,_score,_source.tags,_source.zip_code,_source.complaint_id,_source.issue,_source.date_received,_source.state,...,_source.company_response,_source.company,_source.submitted_via,_source.date_sent_to_company,_source.company_public_response,_source.sub_product,_source.timely,_source.complaint_what_happened,_source.sub_issue,_source.consumer_consent_provided
78308,complaint-public-v2,complaint,3105617,0.0,None,08857,3105617,Closing an account,2018-12-20T12:00:00-05:00,NJ,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2018-12-21T12:00:00-05:00,None,Checking account,Yes,,Funds not received from closed account,Consent not provided
78309,complaint-public-v2,complaint,3091984,0.0,None,934XX,3091984,"Other features, terms, or problems",2018-12-05T12:00:00-05:00,CA,...,Closed with monetary relief,JPMORGAN CHASE & CO.,Web,2018-12-05T12:00:00-05:00,None,General-purpose credit card or charge card,Yes,"On Wednesday, XX/XX/XXXX I called Chas, my XXX...",Other problem,Consent provided
78310,complaint-public-v2,complaint,3133355,0.0,None,923XX,3133355,Problem with a lender or other company chargin...,2019-01-25T12:00:00-05:00,CA,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-01-25T12:00:00-05:00,None,Checking account,Yes,I am not familiar with XXXX pay and did not un...,Transaction was not authorized,Consent provided
78311,complaint-public-v2,complaint,3110963,0.0,None,None,3110963,"Other features, terms, or problems",2018-12-27T12:00:00-05:00,PA,...,Closed with monetary relief,JPMORGAN CHASE & CO.,Web,2018-12-27T12:00:00-05:00,None,General-purpose credit card or charge card,Yes,I have had flawless credit for 30 yrs. I've ha...,Problem with balance transfer,Consent provided
78312,complaint-public-v2,complaint,2001189,0.0,None,None,2001189,Payment to acct not credited,2016-07-06T12:00:00-05:00,MI,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2016-07-07T12:00:00-05:00,None,None,Yes,Roughly 10+ years ago I closed out my accounts...,None,Consent provided


In [6]:
#Assign new column names
df.columns = ['index', 'type', 'id', 'score', 'tags', 'zip_code','complaint_id', 'issue', 'date_received',
       'state', 'consumer_disputed', 'product','company_response', 'company', 'submitted_via',
       'date_sent_to_company', 'company_public_response','sub_product', 'timely',
       'complaint_what_happened', 'sub_issue','consumer_consent_provided']

In [9]:
df['type'].value_counts().sort_values(ascending=False)

complaint    78313
Name: type, dtype: int64

In [10]:
df['tags'].value_counts().sort_values(ascending=False)

Older American                   6103
Servicemember                    3759
Older American, Servicemember    1038
Name: tags, dtype: int64

In [11]:
df['issue'].value_counts().sort_values(ascending=False)

Loan modification,collection,foreclosure                                           9743
Managing an account                                                                7410
Loan servicing, payments, escrow account                                           5360
Problem with a purchase shown on your statement                                    4055
Account opening, closing, or management                                            3879
                                                                                   ... 
Problem with overdraft                                                                1
Was approved for a loan, but didn't receive the money                                 1
Overdraft, savings or rewards features                                                1
Adding money                                                                          1
Money was taken from your bank account on the wrong day or for the wrong amount       1
Name: issue, Length: 154, dtype:

In [12]:
df['consumer_disputed'].value_counts().sort_values(ascending=False)

N/A    35683
No     33107
Yes     9523
Name: consumer_disputed, dtype: int64

In [13]:
df['product'].value_counts().sort_values(ascending=False)

Mortgage                                                                        22725
Checking or savings account                                                     12147
Credit card or prepaid card                                                     10797
Credit card                                                                     10373
Bank account or service                                                          9816
Credit reporting, credit repair services, or other personal consumer reports     4310
Debt collection                                                                  3201
Money transfer, virtual currency, or money service                               1500
Consumer Loan                                                                    1029
Vehicle loan or lease                                                             892
Student loan                                                                      866
Money transfers                                       

In [14]:
df['sub_product'].value_counts().sort_values(ascending=False)

Checking account                              17182
General-purpose credit card or charge card    10504
Other mortgage                                 8447
Conventional fixed mortgage                    4892
Credit reporting                               4172
                                              ...  
Gift or merchant card                             4
Government benefit payment card                   4
ID prepaid card                                   2
Medical debt                                      2
Electronic Benefit Transfer / EBT card            1
Name: sub_product, Length: 72, dtype: int64

In [15]:
df['timely'].value_counts().sort_values(ascending=False)

Yes    78217
No        96
Name: timely, dtype: int64

In [16]:
df['complaint_what_happened'].value_counts().sort_values(ascending=False)

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            57241
Chase has violated 15 USC 1692 by continuing collection activity after a cease and desist and 15 USC 1

In [17]:
df['sub_issue'].value_counts().sort_values(ascending=False)

Credit card company isn't resolving a dispute about a purchase on your statement    2904
Deposits and withdrawals                                                            2759
Company closed your account                                                         2020
Problem using a debit or ATM card                                                   1613
Card was charged for something you did not purchase with the card                   1056
                                                                                    ... 
Problem with direct deposit                                                            1
Confusing or misleading advertising                                                    1
Incorrect information on your report                                                   1
Problem with signing the paperwork                                                     1
Account terms and changes                                                              1
Name: sub_issue, Leng

In [18]:
df['consumer_consent_provided'].value_counts().sort_values(ascending=False)

N/A                     42847
Consent provided        21077
Consent not provided    12285
Other                     991
Consent withdrawn         105
Name: consumer_consent_provided, dtype: int64

In [17]:
null_count_df = pd.DataFrame({'columns':df.columns,
                              'empty_string_count':list(map(lambda column: (df[column] == '').sum(), df.columns)),
                              'null_value_count':list(map(lambda column: (df[column] == np.nan).sum(), df.columns))})
null_count_df

,columns,empty_string_count,null_value_count
0,index,0,0
1,type,0,0
2,id,0,0
3,score,0,0
4,tags,0,0
5,zip_code,0,0
6,complaint_id,0,0
7,issue,0,0
8,date_received,0,0
9,state,0,0


# Initial Data Preprocessing

In [18]:
def clean_text(sentence):
    # Case-folding (Lowercase)
    sentence = sentence.lower()

    # Remove Punctuation, and words containing numbers
    punt_pattern = '[^\w\s]'
    word_with_num_pattern = '\w*\d\w*'
    sentence = re.sub(punt_pattern, '', sentence)
    sentence = re.sub(word_with_num_pattern, '', sentence)
    return sentence

def pos_tagger(text):
    # POS tagger
    pos_sent = []
    blob = TextBlob(text)
    pos_sent = [word for (word,tag) in blob.tags if tag=='NN']
    return " ".join(pos_sent)
    
def lemmatise(text):
    # lemmatise
    lem_sent = []
    doc = nlp(text)
    for token in doc:
        lem_sent.append(token.lemma_)
    return " ".join(lem_sent)

In [19]:
# remove empty value of complaint_what_happened
df_complain_text = df[df['complaint_what_happened']!=""]
df_complain_text

,index,type,id,score,tags,zip_code,complaint_id,issue,date_received,state,...,company_response,company,submitted_via,date_sent_to_company,company_public_response,sub_product,timely,complaint_what_happened,sub_issue,consumer_consent_provided
1,complaint-public-v2,complaint,3229299,0.0,Servicemember,319XX,3229299,Written notification about debt,2019-05-01T12:00:00-05:00,GA,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-05-01T12:00:00-05:00,None,Credit card debt,Yes,Good morning my name is XXXX XXXX and I apprec...,Didn't receive enough information to verify debt,Consent provided
2,complaint-public-v2,complaint,3199379,0.0,None,77069,3199379,"Other features, terms, or problems",2019-04-02T12:00:00-05:00,TX,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-04-02T12:00:00-05:00,None,General-purpose credit card or charge card,Yes,I upgraded my XXXX XXXX card in XX/XX/2018 and...,Problem with rewards from credit card,Consent provided
10,complaint-public-v2,complaint,3233499,0.0,None,104XX,3233499,Incorrect information on your report,2019-05-06T12:00:00-05:00,NY,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-05-06T12:00:00-05:00,None,Other personal consumer report,Yes,Chase Card was reported on XX/XX/2019. However...,Information belongs to someone else,Consent provided
11,complaint-public-v2,complaint,3180294,0.0,None,750XX,3180294,Incorrect information on your report,2019-03-14T12:00:00-05:00,TX,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-03-15T12:00:00-05:00,None,Credit reporting,Yes,"On XX/XX/2018, while trying to book a XXXX XX...",Information belongs to someone else,Consent provided
14,complaint-public-v2,complaint,3224980,0.0,None,920XX,3224980,Managing an account,2019-04-27T12:00:00-05:00,CA,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-04-27T12:00:00-05:00,None,Checking account,Yes,my grand son give me check for {$1600.00} i de...,Funds not handled or disbursed as instructed,Consent provided
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78303,complaint-public-v2,complaint,3094545,0.0,Older American,281XX,3094545,"Advertising and marketing, including promotion...",2018-12-07T12:00:00-05:00,NC,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2018-12-07T12:00:00-05:00,None,General-purpose credit card or charge card,Yes,After being a Chase Card customer for well ove...,Didn't receive advertised or promotional terms,Consent provided
78309,complaint-public-v2,complaint,3091984,0.0,None,934XX,3091984,"Other features, terms, or problems",2018-12-05T12:00:00-05:00,CA,...,Closed with monetary relief,JPMORGAN CHASE & CO.,Web,2018-12-05T12:00:00-05:00,None,General-purpose credit card or charge card,Yes,"On Wednesday, XX/XX/XXXX I called Chas, my XXX...",Other problem,Consent provided
78310,complaint-public-v2,complaint,3133355,0.0,None,923XX,3133355,Problem with a lender or other company chargin...,2019-01-25T12:00:00-05:00,CA,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-01-25T12:00:00-05:00,None,Checking account,Yes,I am not familiar with XXXX pay and did not un...,Transaction was not authorized,Consent provided
78311,complaint-public-v2,complaint,3110963,0.0,None,None,3110963,"Other features, terms, or problems",2018-12-27T12:00:00-05:00,PA,...,Closed with monetary relief,JPMORGAN CHASE & CO.,Web,2018-12-27T12:00:00-05:00,None,General-purpose credit card or charge card,Yes,I have had flawless credit for 30 yrs. I've ha...,Problem with balance transfer,Consent provided


In [20]:
df_complain_text['complaint_clean'] = df_complain_text['complaint_what_happened'].apply(clean_text)
print("Done clean")
df_complain_text['complaint_pos_tagged'] = df_complain_text['complaint_clean'].apply(pos_tagger)
print("Done lemmatisation")
df_complain_text['complaint_lemmatise'] = df_complain_text['complaint_pos_tagged'].apply(lemmatise)
print("POS Tagged")
df_complain_processed = df_complain_text[['complaint_clean','complaint_what_happened','complaint_pos_tagged','complaint_lemmatise']]
df_complain_processed

C:\Users\dxlee\AppData\Local\Temp\ipykernel_10120\712526921.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_complain_text['complaint_clean'] = df_complain_text['complaint_what_happened'].apply(clean_text)


Done clean


C:\Users\dxlee\AppData\Local\Temp\ipykernel_10120\712526921.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_complain_text['complaint_pos_tagged'] = df_complain_text['complaint_clean'].apply(pos_tagger)


Done lemmatisation


C:\Users\dxlee\AppData\Local\Temp\ipykernel_10120\712526921.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_complain_text['complaint_lemmatise'] = df_complain_text['complaint_pos_tagged'].apply(lemmatise)


POS Tagged


,complaint_clean,complaint_what_happened,complaint_pos_tagged,complaint_lemmatise
1,good morning my name is xxxx xxxx and i apprec...,Good morning my name is XXXX XXXX and I apprec...,morning name stop bank cardmember debt verific...,morning name stop bank cardmember debt verific...
2,i upgraded my xxxx xxxx card in and was told ...,I upgraded my XXXX XXXX card in XX/XX/2018 and...,i card agent upgrade date agent information or...,I card agent upgrade date agent information or...
10,chase card was reported on however fraudulent...,Chase Card was reported on XX/XX/2019. However...,card application identity consent credit ident...,card application identity consent credit ident...
11,on while trying to book a xxxx xxxx ticket ...,"On XX/XX/2018, while trying to book a XXXX XX...",book xxxx ticket i offer ticket card i informa...,book xxxx ticket I offer ticket card I informa...
14,my grand son give me check for i deposit it i...,my grand son give me check for {$1600.00} i de...,son deposit chase account fund chase bank acco...,son deposit chase account fund chase bank acco...
...,...,...,...,...
78303,after being a chase card customer for well ove...,After being a Chase Card customer for well ove...,chase card customer decade credit chase airlin...,chase card customer decade credit chase airlin...
78309,on wednesday xxxxxxxx i called chas my xxxx xx...,"On Wednesday, XX/XX/XXXX I called Chas, my XXX...",xxxxxxxx i chas visa credit card provider clai...,xxxxxxxx I cha visa credit card provider claim...
78310,i am not familiar with xxxx pay and did not un...,I am not familiar with XXXX pay and did not un...,i pay risk bank app chase mobile banking i mer...,I pay risk bank app chase mobile banking I mer...
78311,i have had flawless credit for yrs ive had ch...,I have had flawless credit for 30 yrs. I've ha...,i credit credit chase freedom xxxx balance lif...,I credit credit chase freedom xxxx balance lif...


In [21]:
# remove individual "i", "I", "s", "xxxx", "xxxxxxxx"
def clean_specific_unigram(sentence):
    unigram_to_remove = ["i", "I", "s", "xxxxxxxx", "xxxx"]

    # Remove Punctuation, and words containing numbers
    for ug in unigram_to_remove:
        unigram_pattern = f'(\A{ug} )|( {ug} )|( {ug}\Z)'
        sentence = re.sub(unigram_pattern, ' ', sentence)
    return sentence

df_complain_processed['complaint_remove_specific_unigram'] = df_complain_processed['complaint_lemmatise'].apply(clean_specific_unigram)
df_complain_processed


C:\Users\dxlee\AppData\Local\Temp\ipykernel_10120\4036945312.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_complain_processed['complaint_remove_specific_unigram'] = df_complain_processed['complaint_lemmatise'].apply(clean_specific_unigram)


,complaint_clean,complaint_what_happened,complaint_pos_tagged,complaint_lemmatise,complaint_remove_specific_unigram
1,good morning my name is xxxx xxxx and i apprec...,Good morning my name is XXXX XXXX and I apprec...,morning name stop bank cardmember debt verific...,morning name stop bank cardmember debt verific...,morning name stop bank cardmember debt verific...
2,i upgraded my xxxx xxxx card in and was told ...,I upgraded my XXXX XXXX card in XX/XX/2018 and...,i card agent upgrade date agent information or...,I card agent upgrade date agent information or...,card agent upgrade date agent information ord...
10,chase card was reported on however fraudulent...,Chase Card was reported on XX/XX/2019. However...,card application identity consent credit ident...,card application identity consent credit ident...,card application identity consent credit ident...
11,on while trying to book a xxxx xxxx ticket ...,"On XX/XX/2018, while trying to book a XXXX XX...",book xxxx ticket i offer ticket card i informa...,book xxxx ticket I offer ticket card I informa...,book ticket offer ticket card information offe...
14,my grand son give me check for i deposit it i...,my grand son give me check for {$1600.00} i de...,son deposit chase account fund chase bank acco...,son deposit chase account fund chase bank acco...,son deposit chase account fund chase bank acco...
...,...,...,...,...,...
78303,after being a chase card customer for well ove...,After being a Chase Card customer for well ove...,chase card customer decade credit chase airlin...,chase card customer decade credit chase airlin...,chase card customer decade credit chase airlin...
78309,on wednesday xxxxxxxx i called chas my xxxx xx...,"On Wednesday, XX/XX/XXXX I called Chas, my XXX...",xxxxxxxx i chas visa credit card provider clai...,xxxxxxxx I cha visa credit card provider claim...,cha visa credit card provider claim purchase ...
78310,i am not familiar with xxxx pay and did not un...,I am not familiar with XXXX pay and did not un...,i pay risk bank app chase mobile banking i mer...,I pay risk bank app chase mobile banking I mer...,pay risk bank app chase mobile banking mercha...
78311,i have had flawless credit for yrs ive had ch...,I have had flawless credit for 30 yrs. I've ha...,i credit credit chase freedom xxxx balance lif...,I credit credit chase freedom xxxx balance lif...,credit credit chase freedom balance life plen...


# Exploratory data analysis

In [22]:
# unigram frequency
from nltk.tokenize import word_tokenize

text = " ".join(list(df_complain_processed['complaint_remove_specific_unigram']))
uni_tokens = word_tokenize(text)
unigram_df = pd.DataFrame({'unigram':uni_tokens})

unigram_freq_df = unigram_df.groupby('unigram').size().reset_index(name='count').sort_values(by='count', ascending=False)
unigram_freq_df.iloc[:49]

,unigram,count
2195,chase,51793
85,account,41101
3237,credit,32691
1121,bank,26821
1887,card,26054
9573,payment,13967
13294,time,11995
8483,money,11950
2281,check,10031
7709,loan,10008


In [23]:
# bigram
import itertools
from nltk import bigrams
from nltk.tokenize import word_tokenize


text = list(df_complain_processed['complaint_remove_specific_unigram'])
bigrams_2dlist = [list(bigrams(word_tokenize(txt))) for txt in text]
bigrams_flattenlist = list(itertools.chain.from_iterable(bigrams_2dlist))

bigram_df = pd.DataFrame({'bigram':bigrams_flattenlist})

bigram_freq_df = bigram_df.groupby('bigram').size().reset_index(name='count').sort_values(by='count', ascending=False)
bigram_freq_df.iloc[:49]

,bigram,count
68466,"(credit, card)",11032
47633,"(chase, credit)",3490
69320,"(credit, report)",3448
47009,"(chase, account)",3098
47220,"(chase, bank)",3030
71428,"(customer, service)",3026
2849,"(account, chase)",2931
24868,"(bank, account)",2822
2500,"(account, account)",2641
50166,"(check, account)",1918


In [24]:
# trigram
import itertools
from nltk import trigrams
from nltk.tokenize import word_tokenize


text = list(df_complain_processed['complaint_remove_specific_unigram'])
trigrams_2dlist = [list(trigrams(word_tokenize(txt))) for txt in text]
trigrams_flattenlist = list(itertools.chain.from_iterable(trigrams_2dlist))

trigram_df = pd.DataFrame({'trigram':trigrams_flattenlist})
trigram_freq_df = trigram_df.groupby('trigram').size().reset_index(name='count').sort_values(by='count', ascending=False)
trigram_freq_df.iloc[:49]

,trigram,count
153737,"(chase, credit, card)",1950
216140,"(credit, card, account)",822
216351,"(credit, card, company)",814
216317,"(credit, card, chase)",719
216404,"(credit, card, credit)",598
122277,"(card, credit, card)",419
222600,"(credit, report, credit)",375
78005,"(bank, credit, card)",365
12827,"(account, chase, account)",363
216232,"(credit, card, bank)",324


In [25]:
# # word cloud of unigrams
# import numpy as np
# import pandas as pd
# from os import path
# from PIL import Image
# from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

# import matplotlib.pyplot as plt


# # Start with one review:


# from nltk.tokenize import word_tokenize

# text = " ".join(list(df_complain_processed['complaint_remove_specific_unigram']))
# uni_tokens = word_tokenize(text)
# # Create and generate a word cloud image:
# wordcloud = WordCloud().generate(" ".join(uni_tokens))

# # Display the generated image:
# plt.imshow(wordcloud, interpolation='bilinear')
# plt.axis("off")
# plt.show()

# Topic Modelling

In [ ]:
# generate ticket category
# LDA
# NMF
# Top2Vec
# BERTopic

# evaluate which is the best

# Sentiment Analysis

In [ ]:
# generate ticket priority
# based on overall sentiment score, polarity, and topic frequency
# then abc ranking (20-30-50)



# Word Embedding

use one
- tfidf
- word2vec
- doc2vec
- glove
- fasttext

# Model building
- data partition
- machine learning models
    - SVM
    - KNN
    - LogR
    - NB
    - Dtree
    - random forest
- evaluations
    - accuracy, precision, recall, f1-score